<a href="https://colab.research.google.com/github/totminaekaterina/RUSSE-2022-Detoxification/blob/main/detox_ruprompts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip install ruprompts 

  Using cached ruprompts-0.1.4-py3-none-any.whl (29 kB)
  Using cached transformers-4.27.4-py3-none-any.whl (6.8 MB)
  Using cached torch-1.13.1-cp310-cp310-win_amd64.whl (162.6 MB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
  Using cached torchtyping-0.1.4-py3-none-any.whl (17 kB)
  Using cached huggingface_hub-0.13.3-py3-none-any.whl (199 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached regex-2023.3.23-cp310-cp310-win_amd64.whl (267 kB)
     ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.5 MB 1.3 MB/s eta 0:00:03
     ---------------------------------------- 0.0/3.5 MB 487.6 kB/s eta 0:00:08
     - -------------------------------------- 0.1/3.5 MB 930.9 kB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.5 MB 1.1 MB/s eta 0:00:04
     -- ------------------------------------- 0.2/3.5 MB 1.3 MB/s eta 0:00:03
     ----- ---------------------------------- 0.5/3.5 MB

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.15.0 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.
torchaudio 2.0.0 requires torch==2.0.0, but you have torch 1.13.1 which is incompatible.


In [1]:
import torch
print(torch.__version__)

2.0.0


## Training

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda', index=0)

In [7]:
!pip install datasets

     ---------------------------------------- 0.0/468.7 kB ? eta -:--:--
     -- ------------------------------------ 30.7/468.7 kB 1.4 MB/s eta 0:00:01
     ---- -------------------------------- 61.4/468.7 kB 656.4 kB/s eta 0:00:01
     -------- --------------------------- 112.6/468.7 kB 939.4 kB/s eta 0:00:01
     ------------------ ------------------- 225.3/468.7 kB 1.3 MB/s eta 0:00:01
     ----------------------- -------------- 286.7/468.7 kB 1.3 MB/s eta 0:00:01
     -------------------------------------- 468.7/468.7 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/134.3 kB ? eta -:--:--
     -------------------------------------- 134.3/134.3 kB 8.3 MB/s eta 0:00:00
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached dill-0.3.6-py3-none-any.whl (110 kB)
     ---------------------------------------- 0.0/319.8 kB ? eta -:--:--
     ------------------------------------- 319.8/319.8 kB 10.0 MB/s eta 0:00:00
     ---------------------------

Load the backbone:

In [5]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "NLI_for_neg_gen"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Define the prompt format:

In [6]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{premise}<P*20>")

Define the parametrization of trainable embeddings:

In [7]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

Compose prompt format and prompt provider into prompt object and apply it to the model and tokenizer, i.e. add special tokens to the tokenizer and modify the layer of input embeddings of the model:

In [8]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

Preprocess the data:
1. format the data entries with the specified prompt format
2. tokenize the resulting sequences
3. truncate the `truncation_field` if sequence length exceeds `max_tokens`

In [9]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="hypothesis",
    max_tokens=1792,
    truncation_field="premise",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

Loading cached processed dataset at C:\Users\totmi\.cache\huggingface\datasets\csv\default-c2b806a85a59134d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-539fa5e9724b1045.arrow
Loading cached processed dataset at C:\Users\totmi\.cache\huggingface\datasets\csv\default-c2b806a85a59134d\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ddee4bfd774a8447.arrow


Define training arguments:

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Bert",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    eval_steps=5000,
    save_steps=1000,
    logging_steps=5000,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=20000,
    report_to="tensorboard",
    logging_dir="logs",
    seed=1,
)

Choose optimization options:

In [29]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=200,
    num_training_steps=training_args.max_steps,
)

C:\Users\totmi\anaconda3\envs\new_env\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
!pip install tensorboard

     ---------------------------------------- 0.0/5.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/5.6 MB 1.3 MB/s eta 0:00:05
     ---------------------------------------- 0.0/5.6 MB 388.9 kB/s eta 0:00:15
      --------------------------------------- 0.1/5.6 MB 819.2 kB/s eta 0:00:07
     - -------------------------------------- 0.2/5.6 MB 833.5 kB/s eta 0:00:07
     - -------------------------------------- 0.2/5.6 MB 1.1 MB/s eta 0:00:05
     -- ------------------------------------- 0.4/5.6 MB 1.5 MB/s eta 0:00:04
     --- ------------------------------------ 0.5/5.6 MB 1.7 MB/s eta 0:00:04
     ----- ---------------------------------- 0.8/5.6 MB 2.2 MB/s eta 0:00:03
     ------- -------------------------------- 1.1/5.6 MB 2.6 MB/s eta 0:00:02
     ---------- ----------------------------- 1.5/5.6 MB 3.2 MB/s eta 0:00:02
     ----------- ---------------------------- 1.6/5.6 MB 3.4 MB/s eta 0:00:02
     ----------- ---------------------------- 1.6/5.6 MB 3.4 MB/s

In [9]:
import tqdm

Define the callbacks and start training:

In [34]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

KeyboardInterrupt: 

## Inference

Load prompt from the last checkpoint:

In [10]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"checkpoint-20000")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

Run inference:

In [12]:
import pandas as pd

In [14]:
test = pd.read_csv("Train_Only_Sentence.csv", encoding='utf-8-sig', sep=';')

In [15]:
test.head()

,id,text
0,1,Один из создателей сервиса шестисекундных виде...
1,2,Об этом сообщает портал TMZ со ссылкой на исто...
2,3,"По информации издания, утром в воскресенье, 16..."
3,4,После этого сотрудники правоохранительных орга...
4,5,В комнате были найдены также следы наркотиков ...


In [13]:
from datasets import load_dataset

In [17]:
datasets = load_dataset("csv", data_files={"test": "Train_Only_Sentence.csv"}, encoding='utf-8-sig', sep=';')
test_dataset = datasets["test"]

Found cached dataset csv (C:/Users/totmi/.cache/huggingface/datasets/csv/default-6e8d824acc1dca35/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 5

predictions = []
    
for i in tqdm(test_dataset["text"]):
    options = ppln(
        {"premise": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    answer = sorted(options, key=len)[-1]  # get longest answer
    predictions.append(answer)

  0%|          | 9/99999 [00:02<6:03:29,  4.58it/s] C:\Users\totmi\anaconda3\envs\new_env\lib\site-packages\transformers\pipelines\base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████| 99999/99999 [5:53:59<00:00,  4.71it/s]   


In [19]:
pd.DataFrame({"answer": predictions})["answer"].apply(pd.Series).to_csv("Train_Only_Sentence_NLI.csv", index=False)